In [1]:
import requests 
import pandas as pd
import numpy as np
import sys; sys.path
import datetime
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import datetime
from tqdm import tqdm
import random

from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import LinearRegressionBaselineModel, RnnDlModel
from bitcoin_deep_learning.trainer import train
from bitcoin_deep_learning.cross_val import cross_val,se
api = ApiCall()

%load_ext autoreload
%autoreload 2

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [62]:
import requests 
import pandas as pd
import numpy as np
import sys; sys.path
import datetime
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import datetime
from tqdm import tqdm
import random

from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import LinearRegressionBaselineModel, RnnDlModel
from bitcoin_deep_learning.trainer import train
from bitcoin_deep_learning.cross_val import cross_val

%load_ext autoreload
%autoreload 2
from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import LinearRegressionBaselineModel, RnnDlModel
from bitcoin_deep_learning.trainer import train
from bitcoin_deep_learning.cross_val import cross_val,fold_indexes,sequence_indexes
def get_cross_XY(data="train"):
    df = ApiCall().read_local(data='train')
    df = df.drop(columns=["date"])
    # Initializing the variable to return
    X_train_list, Y_train_list, X_test_list,Y_test_list = [], [], [],[]
    # Setting the indexes to cut the df into folds
    start_fold_train, end_fold_train, start_fold_test, end_fold_test = fold_indexes(
        df=df,verbose=verbose)
    # Starting the iteration on folds
    print(len(start_fold_test))
    for i in range(len(start_fold_train)):
        # reinitialise the model between two folds to reset training
        train_fold_df = df.loc[start_fold_train[i]:
                                 end_fold_train[i]].copy().reset_index(drop=True)
        # Setting the indexes to cut the train_fold in regular sequences and targets
        sequence_starts, sequence_stops, target_idx = sequence_indexes(
            df=train_fold_df,verbose=verbose)
        # Initializing the X_train, Y_train
        X_train, Y_train = [], []
        # Starting the iteration on the sequences to create X,Y_train
        for j in range(len(sequence_starts)):

            X_train_seq = np.array(
                train_fold_df.iloc[sequence_starts[j]:sequence_stops[j]],:-1)
            y_train = train_fold_df.iloc[target_idx[j], -1]
            #Converting the little df to np array
            X_train.append(np.array(X_train_seq))
            Y_train.append(np.array(y_train))
        # Converting the list of array to an array>
        Y_train = np.array(Y_train)
        X_train = np.array(X_train)
        
        X_train_list.append(X_train)
        Y_train_list.append(Y_train)

    #Same process as ahead but on the test_fold
        test_fold_df = df.loc[start_fold_test[i]:end_fold_test[i]].copy(
        ).reset_index(drop=True)
        sequence_starts, sequence_stops, target_idx = sequence_indexes(df=test_fold_df,verbose=verbose)
        Y_test,X_test = [],[]
        for j in range(len(sequence_starts)):
            X_test_seq = test_fold_df.iloc[sequence_starts[j]:sequence_stops[j]]
            y_test = test_fold_df.iloc[target_idx[j], -1]
            X_test.append(np.array(X_test_seq))
            Y_test.append(np.array(y_test))
        Y_test = np.array(Y_test)
        X_test = np.array(X_test)

        Y_test_list.append(Y_test)
        X_test_list.append(X_test) 
    
    return (X_train_list, Y_train_list, X_test_list,Y_test_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
(X_train_list, Y_train_list, X_test_list,Y_test_list) = get_cross_XY(data="train")


32


In [68]:
X_train_list[0][1][:,-2]

array([ 8799.12191062,  9208.45991916,  8176.46061576,  6891.93441442,
        7714.39205723,  7575.6111967 ,  8222.75364946,  8677.91052745,
        8537.39320807,  8073.29364922,  8883.25632399,  8519.18610381,
        9483.45790622, 10063.90199802, 10236.5       , 11101.7       ,
       10593.2       , 11230.        , 11403.7       , 10690.4       ,
       10005.        , 10301.1       ,  9813.07      ,  9664.73      ,
       10366.7       , 10725.6       , 10397.9       , 10951.        ,
       11086.4       , 11489.7       , 11512.6       , 11573.3       ,
       10779.9       ,  9965.57      ,  9395.01      ,  9337.55      ,
        8866.        ,  9578.63      ,  9205.12      ,  9194.85      ,
        8269.81      ,  8300.86      ,  8338.35      ,  7916.88      ,
        8223.68      ,  8630.65      ,  8913.47      ,  8929.28      ,
        8721.57      ,  8820.54      ,  8682.62      ,  8494.44      ,
        8210.22      ,  7840.16      ,  7980.12      ,  7162.96      ,
      